# HE Transformer

[HE Transformer](https://github.com/IntelAI/he-transformer) is a homomorphic encryption backend to [Intel's nGraph library/compiler](https://www.intel.com/content/www/us/en/artificial-intelligence/ngraph.html) for AI, allowing us to do deep learning on encrypted data with homomorphic operations. Although both nGraph and HE Transformer are for C++, an [nGraph bridge](https://github.com/tensorflow/ngraph-bridge) is also included, which allows us to use Python and the familiar TensorFlow library with HE Transformer. 

## Installing HE Transformer

We will be using Ubuntu 18.04 (*not* 20.04), to use HE Transformer, as suggested on their GitHub. To install HE Transformer easily, we will use the `he-transformer-setup-instructions.sh` bash file from https://github.com/atkinssamuel/UndergraduateCryptoNetThesisResearch, which can be found [here](https://github.com/atkinssamuel/UndergraduateCryptoNetThesisResearch/blob/master/he-transformer-setup-instructions.sh), with a few modifications. From the original file, we will swap the lines `export HE_TRANSFORMER=$(pwd)` and `cd ~/he-transformer/`, since we need to enter the directory before adding the current directory to the PATH. We will also add two more lines, `make install python_client` and `pip install python/dist/pyhe_client-*.whl` at the end of the file, which set up the Python bindings.

```
#!/bin/bash
# This file runs the appropriate commands to replicate the he-transformer repository located at:
# https://github.com/IntelAI/he-transformer
# You must change the permissions of the file to run. Run: chmod u+x scriptname
# Make sure you run with sudo permissions
cd ~
sudo apt install g++
sudo apt update && sudo apt install -y python3-pip virtualenv python3-numpy python3-dev python3-wheel git unzip wget sudo bash-completion build-essential cmake software-properties-common git wget patch diffutils libtinfo-dev autoconf libtool doxygen graphviz
wget -O - https://apt.llvm.org/llvm-snapshot.gpg.key | sudo apt-key add -
sudo apt-add-repository "deb http://apt.llvm.org/bionic/ llvm-toolchain-bionic-9 main"
sudo apt-get update && sudo apt install -y clang-9 clang-tidy-9 clang-format-9
git clone https://github.com/IntelAI/he-transformer.git
sudo apt-get clean autoclean
sudo apt-get autoremove -y
sudo pip3 install --upgrade pip setuptools virtualenv==16.1.0
sudo -H pip3 install cmake --upgrade
wget https://github.com/bazelbuild/bazel/releases/download/0.25.2/bazel-0.25.2-installer-linux-x86_64.sh
chmod +x ./bazel-0.25.2-installer-linux-x86_64.sh 
sudo bash ./bazel-0.25.2-installer-linux-x86_64.sh 
export PATH=$PATH:~/bin
source ~/.bashrc
cd ~/he-transformer/
export HE_TRANSFORMER=$(pwd)
mkdir build
cd build
sudo cmake .. -DCMAKE_CXX_COMPILER=clang++-9 -DCMAKE_C_COMPILER=clang-9
sudo make install
source $HE_TRANSFORMER/build/external/venv-tf-py3/bin/activate
make install python_client
pip install python/dist/pyhe_client-*.whl
```

Before running the bash script with `sudo bash he-transformer-setup-instructions.sh`, make sure to run `chmod u+x he-transformer-setup-instructions.sh` as it says in the comments of the bash file. This script takes care of installing all of the dependencies necessary for HE Transformer, as well as HE Transformer itself.

**Note: The installation takes very long (took ~9 hours for me), and your computer may not be able to do other tasks while it is installing.**

After the installation is complete, make sure to activate the python environment with ngraph bridge with `source $HE_TRANSFORMER/build/external/venv-tf-py3/bin/activate`. After, run `python $HE_TRANSFORMER/examples/ax.py --backend=HE_SEAL` to verify the installation with a small example of multiplication and addition on encrypted data.

## Testing Basic Operations

**Note**: The code here does *not* run in JupyterLab, since the necessary virtual environment could not be set up inside JupyterLab. Where program execution should result in an output, a screenshot of the output will be provided.

To test the basic operations of homomorphic encryption, addition and subtraction, we can use one of the examples python programs they give, [ax.py](https://github.com/IntelAI/he-transformer/blob/master/examples/ax.py), and modify the main method. We do this since their example program takes care of a lot of the groundwork necessary, such as importing the correct libraries and setting up command line arguments. The import statements should be

In [ ]:
import ngraph_bridge
import argparse
import numpy as np
import tensorflow as tf
from tensorflow.core.protobuf import rewriter_config_pb2

`numpy` and `tensorflow` are the normal libraries we need for machine learning applications. `argparse` is needed to parse command line arguments easily, and `ngraph_bridge` is needed to use he-transformer with tensorflow in Python.

Next, you will have the following function, which helps in parsing string arguments for boolean values (true/false)

In [3]:
def str2bool(v):
    if isinstance(v, bool):
        return v
    if v.lower() in ("on", "yes", "true", "t", "y", "1"):
        return True
    elif v.lower() in ("off", "no", "false", "f", "n", "0"):
        return False
    else:
        raise argparse.ArgumentTypeError("Boolean value expected.")

You will then have

In [5]:
def server_config_from_flags(FLAGS, tensor_param_name):
    rewriter_options = rewriter_config_pb2.RewriterConfig()
    rewriter_options.meta_optimizer_iterations = rewriter_config_pb2.RewriterConfig.ONE
    rewriter_options.min_graph_nodes = -1
    server_config = rewriter_options.custom_optimizers.add()
    server_config.name = "ngraph-optimizer"
    server_config.parameter_map["ngraph_backend"].s = FLAGS.backend.encode()
    server_config.parameter_map["device_id"].s = b""
    server_config.parameter_map[
        "encryption_parameters"].s = FLAGS.encryption_parameters.encode()
    server_config.parameter_map["enable_client"].s = (str(
        FLAGS.enable_client)).encode()
    if FLAGS.enable_client:
        server_config.parameter_map[tensor_param_name].s = b"client_input"

    config = tf.compat.v1.ConfigProto()
    config.MergeFrom(
        tf.compat.v1.ConfigProto(
            graph_options=tf.compat.v1.GraphOptions(
                rewrite_options=rewriter_options)))

    return config

which is a function that takes care of configuring our tensorflow session to run with our encrypted data with he-transformer, rather than regular tensorflow with unencrypted data. This function is one of the major benefits of using their example code, as we can easily get an error free configuration to work with

Lastly, you should have 

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--batch_size", type=int, default=1, help="Batch size")
    parser.add_argument(
        "--enable_client",
        type=str2bool,
        default=False,
        help="Enable the client")
    parser.add_argument(
        "--backend", type=str, default="HE_SEAL", help="Name of backend to use")
    parser.add_argument(
        "--encryption_parameters",
        type=str,
        default="",
        help=
        "Filename containing json description of encryption parameters, or json description itself",
    )

    FLAGS, unparsed = parser.parse_known_args()
    main(FLAGS)

which parses the command line arguments and passes them to the main method, which we will write. First, we start by declaring a 1x1 tensor placeholder variable, which the user provides. This is always needed, since the library was made for machine learning applications, where a user would need to provide an input to make a prediction for. We also define another constant 1x1 tensor. 

In [7]:
def main(FLAGS):
    a = tf.compat.v1.placeholder(
        tf.float32, shape=(1, 1), name="client_parameter_name")

    b = tf.constant(np.ones((1, 1)), dtype=np.float32)

We can then create our configuration, and run the session with said configuration to get both of the values we declared.

In [ ]:
    config = server_config_from_flags(FLAGS, a.name)
    
    with tf.compat.v1.Session(config=config) as sess:
        a_val = sess.run(a, feed_dict={a: np.ones((1, 1))})
        b_val = sess.run(b, feed_dict={a: np.ones((1, 1))})
        print(a_val, b_val)

When running the session, we have to pass the value for the parameter `a`, as that was defined to be a placeholder. Now, we have our initial file done, but we need to make a `.json` file holding our encryption parameters, which we pass in as a command line argument. I call this file `params.json`, which looks like the following

```
{
  "scheme_name": "HE_SEAL",
  "poly_modulus_degree": 8192,
  "security_level": 128,
  "coeff_modulus": [
    60,
    50,
    50,
    50,
    50,
    50,
    60
  ],
  "complex_packing": true
}
```

We declare the scheme name to be used (always HE_SEAL), the polynomial modulus degree, the security level and the small coefficient moduli bit lengths. The polynomial modulus degree must be an integer power of 2, and I choose 8192 here arbitrarily. The security level is 128 by default, and we do not change it here. The small coefficient moduli bit lengths, and the number of them we choose affect the precision of our operations, and the number of levels of operations we can use. Here, we have 7 moduli, so we can have 6 levels of depth in our operations. With our parameters set up, we can run the program using

```
python [python_file_name].py --backend HE_SEAL --encryption_parameters=params.json
```

Remember to put your `params.json` file in the same directory as your Python file to run this. In the case where your parameters file is in another directory, you can alternatively o

```
python [python_file_name].py --backend HE_SEAL --encryption_parameters=/path/to/json/file/params.json
```
Running this should result in

![output1](img/output1.png)

which we know is true, since `a` and `b` are 1x1 tensors with the single element being 1.
If you get an error similar to

![error1](img/error1.png)

then activate the python environment needed using `source $HE_TRANSFORMER/build/external/venv-tf-py3/bin/activate`, and run the Python program again.

Now, we can remove the print statement, and use `vtune -collect system-overview` to measure the total CPU time taken to run the program, which simply encrypts and decrypts twice. The steps on how to set up and use `vtune` are in notebook 2. We then use vtune-gui to check the CPU time it took to run the program. When opening the ouput with vtune-gui, we see  

![vtune1](img/vtune1.png)

near the top. In my case, the program took about 4.1 seconds to run. Now, if we want to find an approximate measure for the time it takes to do one multiplication, we modify our main method to

In [9]:
def main(FLAGS):
    a = tf.compat.v1.placeholder(
        tf.float32, shape=(1, 1), name="client_parameter_name")

    b = tf.constant(np.ones((1, 1)), dtype=np.float32)
    c = a * b
    config = server_config_from_flags(FLAGS, a.name)
    
    with tf.compat.v1.Session(config=config) as sess:
        a_val = sess.run(a, feed_dict={a: np.ones((1, 1))})
        c_val = sess.run(c, feed_dict={a: np.ones((1, 1))})

The method also decrypts two values, and encrypts two values, with the only difference being the extra multiplication. Therefore, any increase in execution time should be due to the the extra operation, multiplication. Using vtune to run and time this program, then finding the increase in execution time, we can find an approximate measure for how long it takes to do a single multiplication. For the multiplication, I get

![vtune2](img/vtune2.png)

Which lets us predict about 0.3 seconds for a single multiplication! We can do these same steps to time addition instead, by replacing `c = a * b` with `c = a + b`. But, addition is so quick (on the order of 1 millisecond), that the uncertainty in the program execution time overshadows the time taken by an addition, and the time with or without addition are too similar to draw any conclusions.